## Matching characteristics of the real data

The attributes of the simulation are provided by the user in a configuration file.  These include:
* Quantities you might read from the headers of the real data, or in reports that the Detector Characterization Lab (DCL) have provided:
 * time steps between each sampled frame (DT)
 * quantum efficiency (QE)
* Quantities drawn from `solid-waffle` fits:
 * gain in e/DN (GAIN)
 * illumination in terms of current in e (ILLUMINATION)
 * linear interpixel capacitance coefficient, $\alpha$ (L_IPC)
 * brighter-fatter coefficients (currently hard-coded in `detector_functions.py`)
 
It may be useful to make some adjustments specific to the data format, for example the FORMAT, but the code currently only supports FORMAT 1. FORMAT is described in detail early in `ScriptInformation.txt`.  Note that many of the recent H4RG-10 data provided by the DCL (e.g., SCA 20829) have FORMAT 4.  In principle, `solid-waffle` results should not care which format is used so long as you specify the correct format in the config file.

In [2]:
# Comparing simulated flats and darks with real ones
import fitsio
import numpy as np
import sys
sys.path.append("../")
import pyirc
# Paths are at the Ohio Supercomputer Center
simdir='/fs/project/PCON0003/ami/simulated_detector/'
datadir='/fs/scratch/PCON0003/cond0007/SCA20829/'
simflat='%s/mar11_sca20829_vissim_diffbfe_flat1.fits'%simdir
#simflat='%s/vissim_bfe_ipc_offdiagp2_flat1.fits'%simdir
#simflat='%s/sca20829_irsim_diffbfe_flat1.fits'%simdir
#simdark='%s/vissim_bfe_ipc_offdiagp2_dark1.fits'%simdir
#simdark='%s/sca20829_irsim_diffbfe_dark2.fits'%simdir
#simdark='%s/sca20829_irsim_diffbfe_dark5_TESTWAVEMODE.fits'%simdir
simdark='%s/mar11_sca20829_vissim_diffbfe_dark1.fits'%simdir
realflat='%s/20191018_95K_1p1m0p1_ch3_1400nm_gr3_filt5_shutter_open_20829_001.fits'%datadir
realdark='%s/20191018_95K_1p1m0p1_ch0_1400nm_gr3_filt5_shutter_closed_20829_001.fits'%datadir
# More efficient to use pyirc tools to do this by superpixel because reading in the whole
# datacube takes up a lot of memory

# Use load_segment from pyirc
# Function to load an image segment
#
# filename = name of the source FITS file
# formatpars = integer describing which type of format to use
#     format 1: H4RG, all data in a single HDU, ramp slope positive (ex. DCL H4RG-18237 data)
# xyrange = list [xmin,xmax,ymin,ymax] (first row/col are zero) -- EXcluding xmax and ymax!
# tslices = list of time slices to use (first is *1*)
# verbose = True or False (use True only for de-bugging)
#
# Returns a 3D array of dimension number tslices, ymax-ymin, xmax-xmin
#
formatpars=4
xyrange=[0,4096,0,4096] # Remember that the first four rows and columns are reference pixels
tslices=[1, 2, 3, 4, 5, 10,20,30,35,40,50,60]
# Recommended True (False defaults to astropy tools, which work but are slow because of the way this script works)
use_fitsio = True

# Get dimensions of output cube
nxuse = xyrange[1]-xyrange[0]
nyuse = xyrange[3]-xyrange[2]
ntslice_use = len(tslices)
realdarkcube = np.zeros((ntslice_use, nyuse, nxuse))

realdarkcube=pyirc.load_segment(realdark, formatpars, xyrange, tslices, verbose=True)

Reading: /fs/scratch/PCON0003/cond0007/SCA20829//20191018_95K_1p1m0p1_ch0_1400nm_gr3_filt5_shutter_closed_20829_001.fits


Some of the useful values we can get from the `solid-waffle` runs on SCA 20829 described in Freudenburg, Givans et al. (https://arxiv.org/pdf/2003.05978.pdf) Table 4 on page 26 of the arxiv pdf version.  These are all for the infrared data.

charge Q =  3051.5 e  Note that to get the `ILLUMINATION` which is e/s, we divide Q by t=2.75s to get 1109.64 e/s

gain g = 1.7285 e/DN

alpha =  0.01379 (we'll take the average and assume symmetry for now)

$\beta_2$g = 2.8147E6 (DN$^{-1})

$\beta_3$g$^2$ = -1.0841E10 (DN$^{-2})

$\beta_4$g$^2$ = 2.3025E10 (DN$^{-3})

I also took a glance at the acceptance test reports available from the detector characterization lab (linked from internal STScI webserver) for 20828 and 20663, and the QE looked to be around 95\% for 20663 and higher, maybe 99\% for 20828.  I didn't see a report for 20829, so let's just go with QE=95\% for the simulation config.  The dark current for 20828 is 0.007 e/s and 0.005 e/s for 20663, so let's go with 0.007 e/s.

One useful thing is to know where the real dark signal level is at and how it compares to the simulated dark signal level

In [3]:
# Find where the real dark signal level is
print(realdarkcube.shape)

# Print out the means, subtract them from 65535 as format 4 signals decrease with time increasing
# and currently the simulated flats will only simulate format 1 with increasing signals with time increasing
for tdx in range(len(tslices)):
    print(np.mean(realdarkcube[tdx,4:-4, 4:-4]))


(12, 4096, 4096)
53927.357412069025
53927.42432333813
53927.82983667055
53928.51856957598
53928.35626479199
53928.26437432263
53929.79686039949
53928.86373193606
53929.200358322865
53929.99635567658
53929.430609935145
53929.99498598112


In [4]:
# Compare to the simulated dark signal level
simdarkcube=np.zeros_like(realdarkcube)
simdarkcube=pyirc.load_segment(simflat, 1, xyrange, tslices, verbose=True)
print(simdarkcube.shape)
# Print out the means
for tdx in range(len(tslices)):
    print(np.mean(simdarkcube[tdx,4:-4, 4:-4]))


Reading: /fs/project/PCON0003/ami/simulated_detector//mar11_sca20829_vissim_diffbfe_flat1.fits
(12, 4096, 4096)
53154.8243246785
51503.2134262588
49868.320008731585
48238.16294738598
46612.7329767019
38555.867283151296
22788.445114645125
7471.660769995041
53.26071779223042
0.0
0.0
0.0


The simulated darks seem a bit different in terms of signal level for the first comparisons I am making: 53927.4 in the real dark and 65477.6 in the simulated dark. This can be adjusted by tuning
```RESET_E: 1.0e2```
in the configuration file that gets passed to the ```simulate_flat.py``` script.

## Finding the BFE kernel values from the `solid-waffle` IR and visible runs
This draws from the code Anna wrote in `plots_fiducial.ipynb` to read in the BFE kernels from the IR and visible outputs.  Then we would like to average the values over superpixels to get a rough input for the simulations.

In [5]:
np.set_printoptions(suppress=True),
outputdir='/users/PCON0003/cond0088/Projects/detectors/sw_outputs/PaperIV_chargediffusion'
ir_file = open('%s/chris_20829vis_fid1_summary.txt'%outputdir)
ir_data = np.loadtxt(ir_file)
ir_file.close()
bfe = np.ma.masked_where(ir_data[:,13:38]==0, ir_data[:,13:38])
out_bfe_arr = bfe.reshape(bfe.shape[0],5,5)
avg_bfe_arr = np.mean(out_bfe_arr,axis=0)
print("sum of avg bfe:", avg_bfe_arr.sum())
# Print in ppm/e for ease of visual comparison
print(np.round(1.E6*avg_bfe_arr,decimals=6))
# Check also how much it deviates from summing to zero
print(np.round(1.E6*avg_bfe_arr,decimals=6).sum())

vis_file = open('%s/chris_20829vis_fid1_visinfo.txt'%outputdir)
vis_data = np.loadtxt(vis_file)
vis_file.close()
vis_bfe = np.ma.masked_where(vis_data[:,:25]==0, vis_data[:,:25])
out_visbfe_arr = vis_bfe.reshape(vis_bfe.shape[0],5,5)
avg_visbfe_arr = np.mean(out_visbfe_arr,axis=0)
print("sum of avg bfe:",avg_visbfe_arr.sum())
print(np.round(1.E6*avg_visbfe_arr,decimals=6))
print(np.round(1.E6*avg_visbfe_arr,decimals=6).sum())
"""ir_cols = ['superX','superY','goodpix','raw_gain','gain_alpha','gain_alphabeta',
                'alphaH','alphaV','betaNL','q_per_t','alphaD','cH','cV',
                'bfe_IR(-2,-2)','bfe_IR(-1,-2)','bfe_IR(0,-2)','bfe_IR(1,-2)','bfe_IR(2,-2)',
                'bfe_IR(-2,-1)', 'bfe_IR(-1,-1)','bfe_IR(0,-1)','bfe_IR(1,-1)','bfe_IR(2,-1)',
                'bfe_IR(-2,0)','bfe_IR(-1,0)','bfe_IR(0,0)','bfe_IR(1,0)','bfe_IR(2,0)',
                'bfe_IR(-2,1)','bfe_IR(-1,1)','bfe_IR(0,1)','bfe_IR(1,1)','bfe_IR(2,1)',
                'bfe_IR(-2,2)','bfe_IR(-1,2)','bfe_IR(0,2)','bfe_IR(1,2)','bfe_IR(2,2)',
                't_intercept','beta2','beta3','beta4']"""
# double-checking how np.reshape maps to indices
test = np.array(('-2,-2','-1,-2','0,-2','1,-2','2,-2','-2,-1','-1,-1','0,-1','1,-1','2,-1','-2,0','-1,0','0,0','1,0','2,0','-2,1','-1,1','0,1','1,1','2,1','-2,2','-1,2','0,2','1,2','2,2'))
print(test.reshape((5,5)))

sum of avg bfe: 3.1714177103068526e-07
[[ 0.002428  0.018179  0.034335  0.017008  0.005557]
 [ 0.010687  0.103002  0.389114  0.11482   0.009735]
 [ 0.023426  0.362651 -1.971387  0.385713  0.031503]
 [ 0.015851  0.118116  0.440877  0.110673  0.010928]
 [ 0.006783  0.016524  0.031761  0.02166   0.007196]]
0.31714
sum of avg bfe: -2.672150955848493e-08
[[ 0.000223 -0.001887  0.012268  0.0083   -0.006043]
 [ 0.010682  0.098918  0.379619  0.111994  0.022616]
 [ 0.016828  0.342355 -2.08724   0.36191   0.01779 ]
 [ 0.00773   0.105579  0.422182  0.100917  0.008805]
 [ 0.003546 -0.000805  0.028231  0.00376   0.004999]]
-0.026723000000000028
[['-2,-2' '-1,-2' '0,-2' '1,-2' '2,-2']
 ['-2,-1' '-1,-1' '0,-1' '1,-1' '2,-1']
 ['-2,0' '-1,0' '0,0' '1,0' '2,0']
 ['-2,1' '-1,1' '0,1' '1,1' '2,1']
 ['-2,2' '-1,2' '0,2' '1,2' '2,2']]


In [6]:
#This cell is me messing around and trying to get a BFE kernel that sums to 0
#input_bfe_ir = avg_bfe_arr - avg_bfe_arr.sum()
input_bfe_ir = np.round(1.E6*avg_bfe_arr,decimals=4)-np.round(1.E6*avg_bfe_arr,decimals=4).sum()/25.
print(np.round(input_bfe_ir,decimals=4).sum())
print(repr(np.round(input_bfe_ir,decimals=4)))

input_bfe_vis = np.round(1.E6*avg_visbfe_arr,decimals=6)-np.round(1.E6*avg_visbfe_arr,decimals=6).sum()/25.
print(input_bfe_vis.sum())
print(repr(np.round(input_bfe_vis,decimals=4)))
test=np.array([[ 0.0013, -0.0008,  0.0133,  0.0094, -0.005 ],
        [ 0.0118,  0.1   ,  0.3807,  0.1131,  0.0237],
        [ 0.0179,  0.3434, -3.0862,  0.363 ,  0.0189],
        [ 0.0088,  0.1066,  0.4233,  0.102 ,  0.0099],
        [ 0.0046,  0.0003,  0.0293,  0.0048,  0.0061]])
(test-test.sum()/25.).sum()

-0.0003000000000001283
masked_array(
  data=[[-0.0103,  0.0055,  0.0216,  0.0043, -0.0071],
        [-0.002 ,  0.0903,  0.3764,  0.1021, -0.003 ],
        [ 0.0107,  0.35  , -1.9841,  0.373 ,  0.0188],
        [ 0.0032,  0.1054,  0.4282,  0.098 , -0.0018],
        [-0.0059,  0.0038,  0.0191,  0.009 , -0.0055]],
  mask=False,
  fill_value=1e+20)
-2.255140518769849e-16
masked_array(
  data=[[ 0.0013, -0.0008,  0.0133,  0.0094, -0.005 ],
        [ 0.0118,  0.1   ,  0.3807,  0.1131,  0.0237],
        [ 0.0179,  0.3434, -2.0862,  0.363 ,  0.0189],
        [ 0.0088,  0.1066,  0.4233,  0.102 ,  0.0099],
        [ 0.0046,  0.0003,  0.0293,  0.0048,  0.0061]],
  mask=False,
  fill_value=1e+20)


-2.0816681711721685e-17

In [16]:
# I put in different kernels for ir and visible flat simulations, ran solid-waffle, and the following examines the outputs
sys.path.append("../flat_simulator/")
from detector_functions import *

simoutdir='%s/out'%simdir
ir_file = open('%s/vis-sim.diffbfe_mar11_summary.txt'%simoutdir)
ir_data = np.loadtxt(ir_file)
ir_file.close()
bfe = np.ma.masked_where(ir_data[:,13:38]==0, ir_data[:,13:38])
out_bfe_arr = bfe.reshape(bfe.shape[0],5,5)
avg_bfe_arr = np.mean(out_bfe_arr,axis=0)
std_bfe_arr = np.std(out_bfe_arr,axis=0)
print("avg ir bfe (after sw run on sim):")
# Print in ppm/e for ease of visual comparison
print(np.round(1.E6*avg_bfe_arr,decimals=6))
print("std dev on the mean for ir bfe:")
print(np.round(1.E6*std_bfe_arr/np.sqrt(32*32-1),decimals=6))
# Check also how much it deviates from summing to zero
print("sum bfe: ",np.round(1.E6*avg_bfe_arr,decimals=6).sum())
print("COMPARE TO SIM INPUT:")
print(np.round(1.E6*np.fliplr(get_bfe_kernel_5x5_ir()),decimals=6))
print("*********************************")

vis_file = open('%s/vis-sim.diffbfe_mar11_visinfo.txt'%simoutdir)
vis_data = np.loadtxt(vis_file)
vis_file.close()
vis_bfe = np.ma.masked_where(vis_data[:,:25]==0, vis_data[:,:25])
out_visbfe_arr = vis_bfe.reshape(vis_bfe.shape[0],5,5)
avg_visbfe_arr = np.mean(out_visbfe_arr,axis=0)
std_visbfe_arr = np.std(out_visbfe_arr,axis=0)
print("avg vis bfe (after sw run on sim):")
print(np.round(1.E6*avg_visbfe_arr,decimals=6))
print("std dev on the mean for ir bfe:")
print(np.round(1.E6*std_bfe_arr/np.sqrt(32*32-1),decimals=6))
print("sum bfe: ",np.round(1.E6*avg_visbfe_arr,decimals=6).sum())
print("COMPARE TO SIM INPUT:")
print(np.round(1.E6*np.fliplr(get_bfe_kernel_5x5_vis()),decimals=6))

avg ir bfe (after sw run on sim):
[[ 0.000521  0.011651  0.018865  0.015453 -0.00314 ]
 [-0.005035  0.098882  0.433164  0.105723  0.003344]
 [ 0.006628  0.358742 -2.036713  0.378611  0.01465 ]
 [-0.005758  0.082366  0.379563  0.105671 -0.009424]
 [-0.005879  0.004043  0.028612 -0.003367 -0.003572]]
std dev on the mean for ir bfe:
[[0.005849 0.006005 0.006029 0.006152 0.006196]
 [0.006174 0.006317 0.006093 0.006095 0.006202]
 [0.006369 0.00598 0.006468 0.006118 0.00642]
 [0.006285 0.006226 0.005959 0.006427 0.006128]
 [0.006099 0.006205 0.006203 0.006048 0.006193]]
sum bfe:  -0.026399000000000818
COMPARE TO SIM INPUT:
[[-0.01    0.0055  0.0216  0.0043 -0.0071]
 [-0.002   0.0903  0.3764  0.1021 -0.003 ]
 [ 0.0107  0.35   -1.9841  0.373   0.0188]
 [ 0.0032  0.1054  0.4282  0.098  -0.0018]
 [-0.0059  0.0038  0.0191  0.009  -0.0055]]
*********************************
avg vis bfe (after sw run on sim):
[[ 0.00974  -0.000745  0.031713  0.000765  0.005244]
 [ 0.006271  0.12435   0.477251  0.12